In [1]:
%%javascript
require(["codemirror/keymap/sublime", "notebook/js/cell", "base/js/namespace"],
    function(sublime_keymap, cell, IPython) {
        cell.Cell.options_default.cm_config.keyMap = 'sublime';
    
        var cells = IPython.notebook.get_cells();
        for(var cl=0; cl< cells.length ; cl++){
            cells[cl].code_mirror.setOption('keyMap', 'sublime');
        }
    }
);

<IPython.core.display.Javascript object>

basically run this bash script to extract all relevant stats from pairs stats

i.e. total nodup pairs, all M-related pairs, M/M pairs, trans-M pairs and trans-M pairs related to unassembled contigs only just in case ...

here is what we'd get in terms of input data:


### FOR CONTIGS ...

a slight modification of the previous thing to enable it work with the contig names ...

one should probably let go of `bash` at this point and do it using a "normal" scripting language - but anyways...



### programm to extract stats from pairs.stats ...

```sh
get_stats () {
    f=$1
    chrom=$2
    dataset=$3
    chrom_cis="$chrom/$chrom\s"
    sample=$(echo $f | cut -f1 -d  "_");
    tot=$(grep "total_nodups" $f | cut -f2);
    cis=$(grep '^cis[^_]' $f | cut -f2);
    trans=$(grep "^trans" $f | cut -f2);
    unmapped=$(grep "total_unmapped" $f | cut -f2);
    dups=$(grep "total_dups" $f | cut -f2);
    allM=$(grep -P "$chrom(\/|\s)" $f | awk '{s+=$2} END {print s}' );
    cisM=$(grep "$chrom_cis" $f | cut -f2);
    transM=$(grep -P "($chrom\/chr[[:alnum:]]*\t|chr[[:alnum:]]*\/$chrom\t)" $f | grep -v $chrom_cis | awk '{s+=$2} END {print s}');
    contigM=$(grep -P "$chrom(\/|\s)" $f | grep -vP "($chrom\/chr[[:alnum:]]*\t|chr[[:alnum:]]*\/$chrom\t)" | grep -v $chrom_cis | awk '{s+=$2} END {print s}');
    [[ -n $allM ]] || allM="0";
    [[ -n $cisM ]] || cisM="0";
    [[ -n $transM ]] || transM="0";
    [[ -n $contigM ]] || contigM="0";
    AA=$(( $cisM+$contigM+$transM ));
    [[ "$AA" == "$allM" ]] || echo "$f $chrom not-equal" >> ~/bbb/log.log;
    awk -v a="$sample" -v b="$tot" -v ba="$cis" -v bb="$trans" -v bc="$unmapped" -v bd="$dups" -v c="$allM" -v d="$cisM" -v e="$transM" -v f="$contigM" -v dt="$dataset" 'BEGIN {print a" "b" "ba" "bb" "bc" "bd" "c" "d" "e" "f" "dt }';
}


for chrom in $(cat ~/hg38.chroms  | cut -f1 -d" " | cut -f2 -d">"); do
    awk 'BEGIN {print "name tot cis trans unmapped dups all_chrom cis_chrom trans_chrom contig_chrom dataset_type" }' > ~/bbb/$chrom.tsv;
    cd /nl/umw_job_dekker/users/ba69w/HiC_Analysis/U54_matrix/results/pairs_library
    for f in  *hg38.dedup.stats; do
        get_stats $f $chrom shallow >> ~/bbb/$chrom.tsv
    done

    cd /nl/umw_job_dekker/users/ba69w/HiC_Analysis/U54_deep/results/pairs_library
    for f in  *hg38.dedup.stats; do
        get_stats $f $chrom deep >> ~/bbb/$chrom.tsv
    done
done
```

### Just testing out some of the "magic" bash commands and grepping super powers ...


```sh
# all involving chrom
chrom="chr1"; f=U54-HFF-plate-FA-DpnII-20180904-R1-T1__hg38.hg38.dedup.stats; grep -P "$chrom(\/|\s)" $f;

# just cis of the chrom
chrom="chr1"; f=U54-HFF-plate-FA-HindIIII-20160226-R2-T1__hg38.hg38.dedup.stats; chrom_cis="$chrom/$chrom\s"; grep "$chrom_cis" $f

# trans_chrom
chrom="chr1";f=U54-HFF-plate-FA-HindIIII-20160226-R2-T1__hg38.hg38.dedup.stats; chrom_cis="$chrom/$chrom\s"; grep -P "($chrom\/chr[[:alnum:]]*\t|chr[[:alnum:]]*\/$chrom\t)" $f | grep -v $chrom_cis

# contig_chrom
chrom="chr1";f=U54-HFF-plate-FA-HindIIII-20160226-R2-T1__hg38.hg38.dedup.stats; chrom_cis="$chrom/$chrom\s"; grep -P "$chrom(\/|\s)" $f | grep -vP "($chrom\/chr[[:alnum:]]*\t|chr[[:alnum:]]*\/$chrom\t)" | grep -v $chrom_cis

# [[ "a" == "a" ]] && echo equal || echo not-equal
```

In [5]:
### Reading data in from the cluster ghpcc ...

chroms = !ssh ghpcc cat /home/sv49w/hg38.chroms | cut -f1 -d" " |cut -f2 -d">"

data = {}
for chrom in chroms:
    dat = !ssh ghpcc cat /home/sv49w/bbb/{chrom}.tsv
    data[chrom] = "\n".join(dat)

In [6]:
# i had to install this beauty https://github.com/matplotlib/ipympl
# to make following to work ...
%matplotlib widget
import ipywidgets as widgets

# %matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import seaborn as sns

import numpy as np

from io import StringIO


# make pandas display entire dataframes
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [7]:
# df = pd.read_csv(StringIO(data),sep=" ")
# # let's parse "name" into cell-type, enzyme, crosslink etc

# # first sanity check : allM = cisM + transM + contigM
# assert (df[["cisM","transM","contigM"]].sum(axis=1) == df["allM"]).all()
# # second one: cis+trans = tot (mapped)
# assert (df[["cis","trans"]].sum(axis=1) == df["tot"]).all()

# df["cis_perc"] = df["cis"]/df["tot"]

In [21]:
#make sure data is defined globally at the very end of the notebook ...

def split_counts_fracs(counts):
    c_fracs = [_c for _c in counts if "frac_" in _c]
    c_counts = [_c for _c in counts if "frac_" not in _c]
    return c_fracs,c_counts

def parse_data(chrom):
    df = pd.read_csv(StringIO(data[chrom]),sep=" ")
    # let's parse "name" into cell-type, enzyme, crosslink etc
    #
    # first one: cis+trans = tot (mapped)
    assert (df[["cis","trans"]].sum(axis=1) == df["tot"]).all()
    # second sanity check : allM = cisM + transM + contigM
    assert (df[["cis_chrom","trans_chrom","contig_chrom"]].sum(axis=1) == df["all_chrom"]).all()

    # df["cis_perc"] = df["cis"]/df["tot"]
    # df["ct_chrom"] = df["cis_chrom"]+df["trans_chrom"]+df["contig_chrom"]
    df["frac_tc"] = df["trans_chrom"]/df["cis_chrom"]
    df["frac_ta"] = df["trans_chrom"]/df["all_chrom"]
    return df


def parse_u54_names(name_split):
    exptype = name_split[0]
    celltype = name_split[1]
    if (exptype == "ENCODE")and(celltype=="HeLa"):
        cross = "FA"
        enzyme = name_split[2]
        cycle = "NS"        
    elif (exptype == "U54")and(celltype == "HFFc6"):
        if name_split[2] in ["p17","p22"]:
            cross = name_split[3]
            enzyme = name_split[4]
        else:
            cross = name_split[2]
            enzyme = name_split[3]
        cycle = "NS"
    elif (exptype == "U54")and(celltype in ["END4DN", "H1ESC4DN", "HFFc64DN","HFFc6"]):
        cross = name_split[3]
        enzyme = name_split[4]
        cycle = "NS"        
    elif celltype in ["END","ESC","END4DN","ESC4DN","H1ESC4DN","ENDMaehr","hEND4DN"]:
        cross = name_split[2]
        enzyme = name_split[3]
        cycle = "NS"
    elif celltype in "HFF":
        cross = name_split[3]
        enzyme = name_split[4]
        cycle = "NS"
    elif celltype == "HelaS3":
        cross = name_split[3]
        enzyme = name_split[4]
        cycle = name_split[2]
    else:
        print("I'm something else - dela with me...")
        print(name_split)
        ret = {}
    if enzyme == "HindIIII":
        enzyme = "HindIII"
    if enzyme not in ['DdeI','DpnII','HindIII','MNase']:
        print("enzyme",enzyme,name_split)
    if cross not in ['DSG','EGS','FA']:
        print("cross",cross,name_split)
    if cycle not in ['NS','G1','M']:
        print("cycle",cycle,name_split)
    ret = {"cell":celltype, "cycle": cycle, "cross": cross, "enzyme": enzyme}
    return pd.Series(ret)

# this should be the same for all of them, but it's ugly this way ...
cell_cycle_cross_enzyme = parse_data("chrM")["name"].str.split("-").apply(parse_u54_names)

In [22]:
for c in cell_cycle_cross_enzyme.columns:
    print(c,cell_cycle_cross_enzyme[c].unique())

cell ['END' 'ESC' 'HelaS3' 'HFF' 'HeLa' 'END4DN' 'ENDMaehr' 'ESC4DN' 'H1ESC4DN'
 'hEND4DN' 'HFFc64DN' 'HFFc6']
cycle ['NS' 'G1' 'M']
cross ['DSG' 'EGS' 'FA']
enzyme ['DdeI' 'DpnII' 'HindIII' 'MNase']


In [25]:
# errors = ddf.groupby(("cross","enzyme","cell")).std()
# means.plot.bar(yerr=errors, ax=ax, capsize=4,logy=True)

fig1 = plt.figure(figsize=(12.5,6),constrained_layout=True)
spec1 = gridspec.GridSpec(ncols=1,nrows=1,figure=fig1)
# # Also make sure the margins and spacing are apropriate
# spec1.update(left=0.05, right=0.95, bottom=0.08, top=0.93, wspace=0.02, hspace=0.03)
# # BUT: this is irrelevant for the saved image, if using bbox_inches='tight'in savefig !
ax1 = fig1.add_subplot(spec1[0,0])


# fig, ax0 = plt.subplots(figsize=(20,3))

style = {'description_width': 'initial'}

counts_selector = widgets.Text(
    value="tot,all_chrom,cis_chrom,trans_chrom,contig_chrom",
#     description='tot,all_chrom,cis_chrom,trans_chrom,contig_chrom:',
    description='tot,all...',
    disabled=False,
    style=style
)

enzyme_selector = widgets.Text(
    value="DdeI,DpnII,HindIII,MNase",
    description='DdeI,DpnII,HindIII,MNase:',
    disabled=False,
    style=style
)


cross_selector = widgets.Text(
    value="DSG,EGS,FA",
    description='DSG,EGS,FA:',
    disabled=False,
    style=style
)


cycle_selector = widgets.Text(
    value="NS,G1,M",
    description='NS,G1,M:',
    disabled=False,
    style=style
)

cells_selector = widgets.Text(
    value='END,ESC,HelaS3,HFF,HeLa,END4DN,ENDMaehr,ESC4DN,H1ESC4DN,hEND4DN,HFFc64DN,HFFc6',
    description='END,ESC,HelaS3,HFF:',
    disabled=False,
    style=style
)


# grouping = ["enzyme_cross_cell_cycle","cross_enzyme_cell_cycle","cell_cross_enzyme_cycle"]


@widgets.interact(
                counts = counts_selector,
                enzymes = enzyme_selector,
                cells = cells_selector,
                cycle = cycle_selector,
                cross = cross_selector,
                log=True,
                normalized=True,
                grouping = ["enzyme_cross_cell_cycle","cross_enzyme_cell_cycle","cell_cross_enzyme_cycle"],
                chrom = list(data.keys()))
def update(counts,enzymes,cells,cycle,cross,log,normalized,grouping,chrom):
    
    # this is just to make us able to change chroms
    df = parse_data(chrom)
    
    ax1.clear()
    counts = counts.split(",")
    enzymes = enzymes.split(",")
    cells = cells.split(",")
    cycle = cycle.split(",")
    cross = cross.split(",")
    
    c1 = cell_cycle_cross_enzyme["enzyme"].isin(enzymes)
    c2 = cell_cycle_cross_enzyme["cell"].isin(cells)
    c3 = cell_cycle_cross_enzyme["cycle"].isin(cycle)
    c4 = cell_cycle_cross_enzyme["cross"].isin(cross)
    
    ccce = cell_cycle_cross_enzyme[c1&c2&c3&c4]
    
    if normalized:
        c_fracs,c_counts = split_counts_fracs(counts)
        df_norm = df[c_counts]/df[["tot"]].values
        loc_df = pd.merge(df_norm,ccce,left_index=True,right_index=True)
        loc_df = pd.merge(loc_df,df[c_fracs],left_index=True,right_index=True)
    else:
        df_norm = df[counts]
        loc_df = pd.merge(df_norm,ccce,left_index=True,right_index=True)
    
    
    grp = grouping.split("_")
    mmeans = loc_df.groupby(grp).mean()
    mmeans.plot.bar( ax=ax1, capsize=4,logy=log)
    ax1.set_ylabel("# of pairs")
    
#     print(enzymes)
#     print(cells)
#     print(cycle)
#     print(cross)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Text(value='tot,all_chrom,cis_chrom,trans_chrom,contig_chrom', description='tot,all...',…

the fact that $cis_{chrom} + trans_{chrom}$ is not even close to a ${const}$ is easy to understand because we are over-counting trans data ...

In [29]:

# errors = ddf.groupby(("cross","enzyme","cell")).std()
# means.plot.bar(yerr=errors, ax=ax, capsize=4,logy=True)

fig2 = plt.figure(figsize=(7,5),constrained_layout=True)
spec2 = gridspec.GridSpec(ncols=1,nrows=1,figure=fig2)
# # Also make sure the margins and spacing are apropriate
# spec1.update(left=0.05, right=0.95, bottom=0.08, top=0.93, wspace=0.02, hspace=0.03)
# # BUT: this is irrelevant for the saved image, if using bbox_inches='tight'in savefig !
ax2 = fig2.add_subplot(spec2[0,0])


# fig, ax0 = plt.subplots(figsize=(20,3))

style = {'description_width': 'initial'}

xy_selector = widgets.Text(
    value="cis_chrom,trans_chrom",
    description='tot,all_chrom...',
    disabled=False,
    style=style
)

enzyme_selector = widgets.Text(
    value="DdeI,DpnII,HindIII,MNase",
    description='DdeI,DpnII,HindIII,MNase:',
    disabled=False,
    style=style
)


cross_selector = widgets.Text(
    value="DSG,EGS,FA",
    description='DSG,EGS,FA:',
    disabled=False,
    style=style
)


cycle_selector = widgets.Text(
    value="NS,G1,M",
    description='NS,G1,M:',
    disabled=False,
    style=style
)

cells_selector = widgets.Text(
    value='END,ESC,HelaS3,HFF,HeLa,END4DN,ENDMaehr,ESC4DN,H1ESC4DN,hEND4DN,HFFc64DN,HFFc6',
    description='END,ESC,HelaS3,HFF:',
    disabled=False,
    style=style
)


# grouping = ["enzyme_cross_cell_cycle","cross_enzyme_cell_cycle","cell_cross_enzyme_cycle"]


@widgets.interact(
                xy = xy_selector,
                enzymes = enzyme_selector,
                cells = cells_selector,
                cycle = cycle_selector,
                cross = cross_selector,
                log=True,
                normalized=True,
                chrom = list(data.keys()))
def update(xy,enzymes,cells,cycle,cross,log,normalized,chrom):
    
    df = parse_data(chrom)
    
    ax2.clear()
    x,y = xy.split(",")
    enzymes = enzymes.split(",")
    cells = cells.split(",")
    cycle = cycle.split(",")
    cross = cross.split(",")
    
    c1 = cell_cycle_cross_enzyme["enzyme"].isin(enzymes)
    c2 = cell_cycle_cross_enzyme["cell"].isin(cells)
    c3 = cell_cycle_cross_enzyme["cycle"].isin(cycle)
    c4 = cell_cycle_cross_enzyme["cross"].isin(cross)
    
    ccce = cell_cycle_cross_enzyme[c1&c2&c3&c4]

    
    if normalized:
        df_x = df[[x]]/df[["tot"]].values if "frac_" not in x else df[[x]]
        df_y = df[[y]]/df[["tot"]].values if "frac_" not in y else df[[y]]
        loc_df = pd.merge(df_x,ccce,left_index=True,right_index=True)
        loc_df = pd.merge(loc_df,df_y,left_index=True,right_index=True)
    else:
        df_norm = df[[x,y]]
        loc_df = pd.merge(df_norm,ccce,left_index=True,right_index=True)
        
    sp = sns.scatterplot(x=x,y=y,hue="cross",size="enzyme",data=loc_df,ax=ax2)
        
    x_span = loc_df[x].max() - loc_df[x].min()
    y_span = loc_df[y].max() - loc_df[y].min()

    dx = 0.01*x_span
    dy = 0.01*y_span

    if log:
        ax2.set_xlim((loc_df[x].min()*0.9,loc_df[x].max()*1.01))
        ax2.set_ylim((loc_df[y].min()*0.9,loc_df[y].max()*1.01))
        sp.set(xscale="log")
        sp.set(yscale="log")
    else:
        ax2.set_xlim((loc_df[x].min()-dx,loc_df[x].max()+dx))
        ax2.set_ylim((loc_df[y].min()-dy,loc_df[y].max()+dy))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Text(value='cis_chrom,trans_chrom', description='tot,all_chrom...', style=DescriptionSty…

In [13]:

# errors = ddf.groupby(("cross","enzyme","cell")).std()
# means.plot.bar(yerr=errors, ax=ax, capsize=4,logy=True)

fig3 = plt.figure(figsize=(4,7),constrained_layout=True)
spec3 = gridspec.GridSpec(ncols=1,nrows=3,figure=fig3)
# # Also make sure the margins and spacing are apropriate
# spec1.update(left=0.05, right=0.95, bottom=0.08, top=0.93, wspace=0.02, hspace=0.03)
# # BUT: this is irrelevant for the saved image, if using bbox_inches='tight'in savefig !
ax31 = fig3.add_subplot(spec3[0,0])
ax32 = fig3.add_subplot(spec3[1,0])
ax33 = fig3.add_subplot(spec3[2,0])


style = {'description_width': 'initial'}

xy_selector = widgets.Text(
    value="cis_chrom,trans_chrom",
    description='tot,all_chrom...',
    disabled=False,
    style=style
)

enzyme_selector = widgets.Text(
    value="DdeI,DpnII,HindIII,MNase",
    description='DdeI,DpnII,HindIII,MNase:',
    disabled=False,
    style=style
)


cross_selector = widgets.Text(
    value="DSG,EGS,FA",
    description='DSG,EGS,FA:',
    disabled=False,
    style=style
)


cycle_selector = widgets.Text(
    value="NS,G1,M",
    description='NS,G1,M:',
    disabled=False,
    style=style
)

cells_selector = widgets.Text(
    value='END,ESC,HelaS3,HFF,HeLa,END4DN,ENDMaehr,ESC4DN,H1ESC4DN,hEND4DN,HFFc64DN,HFFc6',
    description='END,ESC,HelaS3,HFF:',
    disabled=False,
    style=style
)


# grouping = ["enzyme_cross_cell_cycle","cross_enzyme_cell_cycle","cell_cross_enzyme_cycle"]


@widgets.interact(
                xy = xy_selector,
                enzymes = enzyme_selector,
                cells = cells_selector,
                cycle = cycle_selector,
                cross = cross_selector,
                log=True,
                normalized=True,
                chrom = list(data.keys()))
def update(xy,enzymes,cells,cycle,cross,log,normalized,chrom):
    
    df = parse_data(chrom)
    
    ax31.clear()
    ax32.clear()
    ax33.clear()
    enzymes = enzymes.split(",")
    cells = cells.split(",")
    cycle = cycle.split(",")
    cross = cross.split(",")
    
    c1 = cell_cycle_cross_enzyme["enzyme"].isin(enzymes)
    c2 = cell_cycle_cross_enzyme["cell"].isin(cells)
    c3 = cell_cycle_cross_enzyme["cycle"].isin(cycle)
    c4 = cell_cycle_cross_enzyme["cross"].isin(cross)
    
    ccce = cell_cycle_cross_enzyme[c1&c2&c3&c4]
    
    x,y1,y2,y3 = "trans","trans_chrom","cis_chrom","all_chrom"
    if normalized:
        df_norm = df[[x,y1,y2,y3]]/df[["tot"]].values
        loc_df = pd.merge(df_norm,ccce,left_index=True,right_index=True)
    else:
        df_norm = df[[x,y1,y2,y3]]
        loc_df = pd.merge(df_norm,ccce,left_index=True,right_index=True)
    sp1 = sns.scatterplot(x=x,y=y1,hue="cross",size="enzyme",data=loc_df,ax=ax31)
    sp2 = sns.scatterplot(x=x,y=y2,hue="cross",size="enzyme",data=loc_df,ax=ax32)
    sp3 = sns.scatterplot(x=x,y=y3,hue="cross",size="enzyme",data=loc_df,ax=ax33)
    
    x_span = df_norm[x].max() - df_norm[x].min()
    y_span = df_norm[[y1,y2,y3]].max().max() - df_norm[[y1,y2,y3]].min().min()

    dx = 0.01*x_span
    dy = 0.01*y_span

    if log:
        ax31.set_xlim((df_norm[x].min()*0.9,df_norm[x].max()*1.01))
        ax31.set_ylim((df_norm[[y1,y2,y3]].min().min()*0.9,df_norm[[y1,y2,y3]].max().max()*1.01))
        ax32.set_xlim((df_norm[x].min()*0.9,df_norm[x].max()*1.01))
        ax32.set_ylim((df_norm[[y1,y2,y3]].min().min()*0.9,df_norm[[y1,y2,y3]].max().max()*1.01))
        ax33.set_xlim((df_norm[x].min()*0.9,df_norm[x].max()*1.01))
        ax33.set_ylim((df_norm[[y1,y2,y3]].min().min()*0.9,df_norm[[y1,y2,y3]].max().max()*1.01))
        sp1.set(xscale="log",yscale="log")
        sp2.set(xscale="log",yscale="log")
        sp3.set(xscale="log",yscale="log")
    else:
        ax31.set_xlim((df_norm[x].min()-dx,df_norm[x].max()+dx))
        ax31.set_ylim((df_norm[[y1,y2,y3]].min().min()-dy,df_norm[[y1,y2,y3]].max().max()+dy))
        ax32.set_xlim((df_norm[x].min()-dx,df_norm[x].max()+dx))
        ax32.set_ylim((df_norm[[y1,y2,y3]].min().min()-dy,df_norm[[y1,y2,y3]].max().max()+dy))
        ax33.set_xlim((df_norm[x].min()-dx,df_norm[x].max()+dx))
        ax33.set_ylim((df_norm[[y1,y2,y3]].min().min()-dy,df_norm[[y1,y2,y3]].max().max()+dy))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Text(value='cis_chrom,trans_chrom', description='tot,all_chrom...', style=DescriptionSty…